# Descrição do Notebook

Projeto com a ideia de analises o impacto de diferentes centralidade no algoritmo Cuthill McKee

## Algoritmos Base: Cuthill McKee e Reverse Cuthill McKee

## Utilizada também a lib NetworkX de python

## Centralidade utilizadas:





...

In [1]:
#imports do projeto

#### Import to read the files dict 
import os
from os import listdir
from os.path import isfile, join
from os import error

import datetime
import numpy as np
from numpy import arange

import math
import csv

### Import to run the graph lib
import networkx as nx
from networkx.utils import cuthill_mckee_ordering
from networkx.utils import reverse_cuthill_mckee_ordering

from collections import defaultdict


## Auxiliary Functions

In [2]:
###### measure bandwidth  ########

mypath = "..."

# list_path = rf.readFilesInDict(mypath, '.mtx')

A = [[1,4], [0,2,4], [1,3], [2,4,5], [0,1,3], [3]]
f1 = [2, 3, 4, 1, 5, 6]

B = [[1], [0, 2, 3], [1], [1, 4], [3]]
f2 = [3, 1, 2, 5, 4]

def dif(p, q):
    return abs(p-q)
        
def measureBand_old(A, f):
    width = 0
    for v in range(len(A)):
        for adj in A[v]:
            temp = dif(f[v], f[adj])
            if temp > width:
                width = temp
            
    return width

###### measure bandwidth  ########

def labels(R):
    f = defaultdict(int)
    label = 1
    for node in R:
        f[node] = label
        label=label+1
    return f


def measureBand(adj_dict, F):
    width = 0
    for key in adj_dict:
        for node in adj_dict[key]:
            temp = abs(F[key] - F[node])
            if temp > width:
                width = temp
            
    return width

In [3]:
#### Function to read the files dict ####

def readFilesInDict(path, extension):
    onlymtxComp = [os.path.join(path, file) for file in os.listdir(path) if file.endswith(extension)]
    return onlymtxComp

In [4]:
#### function to read the instance and data struct ####

def load_instance(filename):
    edges = []
    neighbours = defaultdict(list)
    neigh = defaultdict(list)
    flag = True
    f = open(filename, 'r')
    for line in f:

        if flag == True:
            nnodes, nedges = [int(x) for x in line.split()]
            flag = False
        else:
            e1, e2 = [int(x) for x in line.split()]

            neigh[e1].append(e2)
            if e1 != e2:
                edges.append((min(e1, e2), max(e1, e2)))
                neighbours[e1].append(e2)
                neighbours[e2].append(e1)
            else:
                nedges = nedges - 1
    f.close()

    f = []
    
    for v in neighbours:
        f.append(v)

    nodes = []

    for v in neighbours:
        nodes.append(v)

    lista_adj = []

    for n in nodes:
        lista_adj.append(neighbours[n])

    matrix = []

    for key in neighbours:
        line = [0]*nnodes
        for element in neighbours[key]:
            line[element-1] = 1
        matrix.append(line)
    
    return nnodes, nedges, edges, neighbours, lista_adj, matrix

def print_instance(qtd_nodes, qtd_edges, edges, neighbours):
    print(str(qtd_nodes)+" "+str(qtd_edges))

    for e in edges:
        print(str(e[0])+" "+str(e[1]))

    for i in neighbours:
        print(neighbours[i])

    print(neighbours)

# nome_arquivo = "/home/joaovitor/Documents/TCC/research-cefet/bandwidth-reduction/data/494_bus.mtx"

# nnodes, nedges, edges, neighbours, lista_adj, matrix = load_instance(nome_arquivo)
# f = [None]*len(neighbours)

# for line in range(len(matrix)):
#     print(matrix[line])

# print("nnodes: ",nnodes)
# print("nedges: ",nedges)
# print("edges: ",edges) 
# print("neighbours: ",neighbours)
# print("lista_adj: ",lista_adj)


## Extracting DATA

## Handling with lib NetworkX


In [5]:
#HEURISTIC SETTING

# HEURISTIC BIGGEST DEGREE
def biggest_degree(G):
    return max(G, key=G.degree)

# HEURISTIC SMALLEST DEGREE
def smallest_degree(G):
    return min(G, key=G.degree)


# HEURISTIC BIGGEST EIGENVECTOR
def smallest_eigenvector(G):
    centrality = nx.eigenvector_centrality(G, max_iter=600)
    return max(centrality, key=centrality.get)

# HEURISTIC SMALLEST EIGENVECTOR
def biggest_eigenvector(G):
    centrality = nx.eigenvector_centrality(G, max_iter=600)
    return min(centrality, key=centrality.get)

# HEURISTIC BIGGEST KATZ
def biggest_katz(G):
    centrality = nx.katz_centrality(G, max_iter=10000)
    return max(centrality, key=centrality.get)

# HEURISTIC SMALLEST KATZ
def smallest_katz(G):
    centrality = nx.katz_centrality(G, max_iter=10000)
    return min(centrality, key=centrality.get)

# HEURISTIC BIGGEST CLOSENESS
def biggest_closeness(G):
    centrality = nx.closeness_centrality(G)
    return max(centrality, key=centrality.get)

# HEURISTIC SMALLEST CLOSENESS
def smallest_closeness(G):
    centrality = nx.closeness_centrality(G)
    return min(centrality, key=centrality.get)

# HEURISTIC BIGGEST HARMONIC
def biggest_harmonic(G):
    centrality = nx.harmonic_centrality(G)
    return max(centrality, key=centrality.get)

# HEURISTIC SMALLEST HARMONIC
def smallest_harmonic(G):
    centrality = nx.harmonic_centrality(G)
    return min(centrality, key=centrality.get)

#HEURISTIC BIGGEST BETWEENNESS
def biggest_betweenness(G):
    centrality = nx.betweenness_centrality(G)
    return max(centrality, key=centrality.get)

# HEURISTIC SMALLEST BETWEENNESS
def smallest_betweenness(G):
    centrality = nx.betweenness_centrality(G)
    return min(centrality, key=centrality.get)




#FUNCTION TO GET BANDWIDTH 
def bandwidth(G):
    '''Calculate the bandwidth'''
    A = nx.adjacency_matrix(G)
    x, y = np.nonzero(A)
    
    w = (y - x).max() + (x - y).max() + 1
    return w

#INIT A GRAPH
def init_graph(edges):
    G = nx.Graph()
    G.add_edges_from(edges)
    return G

# BANDWIDTH CUTHILL MCKEE
def cuthill(G, heuristic):
    rcm = list(cuthill_mckee_ordering(G, heuristic=heuristic))
    adj_matrix = nx.adjacency_matrix(G, nodelist=rcm)
    x, y = np.nonzero(adj_matrix)
    return (y - x).max() + (x - y).max() + 1

# BANDWIDTH REVERSE CUTHILL MCKEE
def reverse_cuthill(G, heuristic):
    rcm = list(reverse_cuthill_mckee_ordering(G, heuristic=heuristic))
    adj_matrix = nx.adjacency_matrix(G, nodelist=rcm)
    x, y = np.nonzero(adj_matrix)
    return (y - x).max() + (x - y).max() + 1


In [6]:


nnodes, nedges, edges, neighbours, lista_adj, matrix = load_instance(list_path[0])

A = init_graph(edges)

# w2 = cuthill(G, biggest_closeness)
# print(f"low bandwidth cuthill: {w2}")
    
# w3 = cuthill(G, smallest_closeness)
# print(f"low bandwidth reverse cuthill: {w3}")

# w4 = cuthill(G, smallest_katz)
# print(f"low bandwidth cuthill: {w2}")
    
w5 = cuthill(A, biggest_katz)
print(f"low bandwidth reverse cuthill: {w3}")

NameError: name 'list_path' is not defined

## Main

In [ ]:
#Main cell that call the tested algorithms

main_path = "../data"
list_path = readFilesInDict(main_path, ".mtx")

heuristics = [ biggest_degree, smallest_degree, smallest_eigenvector, biggest_eigenvector, biggest_katz, smallest_katz,
                     biggest_closeness, smallest_closeness, biggest_harmonic, smallest_harmonic, biggest_betweenness, smallest_betweenness ]

with open('resultados.csv', mode='a+') as csv_file:

    fieldnames = ['instance','timestamp', 'original_band', 'biggest_degree', 'smallest_degree', 'smallest_eigenvector', 'biggest_eigenvector', 'biggest_katz', 'smallest_katz',
                     'biggest_closeness', 'smallest_closeness', 'biggest_harmonic', 'smallest_harmonic', 'biggest_betweenness', 'smallest_betweenness' ]

    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    
    

    for instance_path in list_path:
        

            instance_name = instance_path.replace(main_path, "").replace("/", "").replace(".mtx", "")

            nnodes, nedges, edges, neighbours, lista_adj, matrix = load_instance(instance_path)
            
            print("#########################", instance_name ,"############################")
            
            #initial vars
            solutions = []
            G = init_graph(edges)
            original_w = bandwidth(G)
            
            
            heuristics_name = [ 'biggest_degree', 'smallest_degree', 'biggest_eigenvector', 'smallest_eigenvector','biggest_katz', 'smallest_katz',
                               'biggest_closeness', 'smallest_closeness', 'biggest_harmonic', 'smallest_harmonic','biggest_betweenness', 'smallest_betweenness' ]
            
            solutions.append(original_w)
            print(instance_name)
            
    
            for h in heuristics:
                
                #w_cm = cuthill(G, h)
                #print(f"low bandwidth cuthill: {w_rcm}")
                
                try:
                    w_rcm = reverse_cuthill(G, h)
                    solutions.append(w_rcm)
                except:
                    solutions.append("error")
                

            

            writer.writerow({'instance': instance_name,
                             'timestamp': datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"),
                             'original_band': solutions[0],
                             'biggest_degree': solutions[1],
                             'smallest_degree': solutions[2],
                             'biggest_eigenvector': solutions[3],
                             'smallest_eigenvector': solutions[4],
                             'biggest_katz': solutions[5],
                             'smallest_katz': solutions[6],
                             'biggest_closeness': solutions[7],
                             'smallest_closeness': solutions[8],
                             'biggest_harmonic': solutions[9],
                             'smallest_harmonic': solutions[10],
                             'biggest_betweenness': solutions[11],
                             'smallest_betweenness': solutions[12]
                            })
            

In [ ]:

with open('resultados.csv', mode='a+') as csv_file:

    fieldnames = ['instance','timestamp', 'constructive', 'constructive random',  'cuchill mckee', 'reverse cuchill mckee' ]

    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    
   

    for instance_path in list_path:

        instance_name = instance_path.replace(mypath, "").replace("/", "").replace(".mtx", "")

        nnodes, nedges, edges, neighbours, lista_adj, matrix = ri.load_instance(instance_path)
        

        
        
        writer.writerow({'instance': instance_name,
                         'timestamp': datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"),
                         'constructive': w1,
                         'constructive random': w2,
                         'cuchill mckee': w3,
                         'reverse cuchill mckee': w4})

In [ ]:
import pandas as pd
import seaborn as sn

#data = pd.read_csv("data.csv")

# análise de correlação
correlation = data.corr()

# plot da matriz de correlação

plot = sn.heatmap(correlation, annot = True, fmt=".1f", linewidths=.6)

: 